In [6]:
import json
with open('output.json', 'r') as file:
    # Load JSON data from the file
    companies_data = json.load(file)

companies_data

[{'Filing Type': 'Corporation - Business - Foreign',
  'Status': 'Active',
  'Standing - AR': 'Good',
  'Standing - RA': 'Good',
  'Standing - Other': 'Good',
  'Formed In': 'NORTH CAROLINA',
  'Term of Duration': 'Perpetual',
  'Initial Filing Date': '05/30/2018',
  'Principal Address': '4043 PIEDMONT PARKWAY\nHIGH POINT, NC 27265-9402',
  'Mailing Address': '615 S COLLEGE ST\nFL 9\nTAX DEPT\nCHARLOTTE, NC 28202-3355',
  'AR Due Date': '05/15/2025',
  'Commercial Registered Agent': 'REGISTERED AGENT SOLUTIONS, INC.\n1709 N 19TH ST STE 3\nBISMARCK, ND  58501-2121',
  'COMPANY': 'GXO LOGISTICS SUPPLY CHAIN, INC.'},
 {'Filing Type': 'Limited Liability Company - Business - Foreign',
  'Status': 'Active',
  'Standing - AR': 'Good',
  'Standing - RA': 'Good',
  'Standing - Other': 'Good',
  'Formed In': 'DELAWARE',
  'Term of Duration': 'Perpetual',
  'Initial Filing Date': '11/26/2019',
  'Principal Address': 'ONE WORLD TRADE CENTER\n41ST FLOOR\nNEW YORK, NY 10007',
  'Mailing Address': '1

In [13]:
import networkx as nx
G = nx.Graph() # Person or Company that is an agent / owner
for data in companies_data:
    company_name = data["COMPANY"]
    G.add_node(company_name, type="Company")
    if "Commercial Registered Agent" in data:
        cr_agent = data["Commercial Registered Agent"].split("\n")[0]
        G.add_node(cr_agent, type="Person")
        G.add_edges_from([(company_name, cr_agent),])
    if "Registered Agent" in data:
        r_agent = data["Registered Agent"].split("\n")[0]
        G.add_node(r_agent, type="Person")
        G.add_edges_from([(company_name, r_agent),])
    if "Owner Name" in data:
        owner = data["Owner Name"]
        G.add_node(owner, type="Person")
        G.add_edges_from([(company_name, owner),])
    elif "Owners" in data:
        owner_1 = data["Owners"].split("\n")[0]
        owner_2 = data[""].split("\n")[0]
        G.add_node(owner_1, type="Person")
        G.add_node(owner_2, type="Person")
        edges = [(owner_1, company_name), (owner_2, company_name),]
        G.add_edges_from(edges)

CRAIG NICHOLAS THOMAS
xxx
AMANDA KAY THOMAS
---------
CHARLES DOSH
xxx
SHAWNA DOSH
---------
Corey K Getzlaff
xxx
Cindy J Getzlaff
---------
